In [1]:
import os
import datetime
import winsound
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [2]:
df2=pd.read_csv('data2/table.csv')

In [3]:
df2['BeginTimeH']=[datetime.datetime.strptime(str(time), '%H:%M:%S').strftime('%H') for time in df2['BeginTime']]
df2['BeginTimeM']=[datetime.datetime.strptime(str(time), '%H:%M:%S').strftime('%M') for time in df2['BeginTime']]
df2['DurationH']=[datetime.datetime.strptime(str(time), '%H:%M:%S').strftime('%H') for time in df2['Duration']]
df2['DurationM']=[datetime.datetime.strptime(str(time), '%H:%M:%S').strftime('%M') for time in df2['Duration']]

In [4]:
df4=pd.concat([df2.iloc[:,74:78],df2.iloc[:,2:74]],axis=1)

In [5]:
group=df4.groupby('Activity')

In [6]:
groups = {}
for k, g in group:
    groups[k]=g  
for i in groups:
    groups[i]=pd.DataFrame(groups[i])

In [7]:
meal=groups['Meal_Preparation'].iloc[:,[0,2,75]]
r1sleep=groups['R1_Sleep'].iloc[:,[0,2,75]]
r2sleep=groups['R2_Sleep'].iloc[:,[0,2,75]]
r1toilet=groups['R1_Bed_to_Toilet'].iloc[:,[0,2,75]]
r2toilet=groups['R2_Bed_to_Toilet'].iloc[:,[0,2,75]]
# stairs=groups['up&downstairs'].iloc[:,[2,3,75]]

In [8]:
sleep=r1sleep.append(r2sleep)
sleep.reset_index(inplace=True)
sleep=sleep.sort_values(by = 'index',ascending=True)
sleep=sleep.set_index(['index'],drop=True)
# sort_values(by = "Datetime",ascending=True)

In [9]:
toilet=r1toilet.append(r2toilet)
toilet.reset_index(inplace=True)
toilet=toilet.sort_values(by = 'index',ascending=True)
toilet=toilet.set_index(['index'],drop=True)

In [10]:
# stairs=stairs.append(stairs)
# stairs.reset_index(inplace=True)
# stairs=stairs.sort_values(by = 'index',ascending=True)
# stairs=stairs.set_index(['index'],drop=True)

In [11]:
import warnings
warnings.filterwarnings("ignore")

meal['label']='Meal_normal'
for i,t in enumerate(meal['DurationH']):
    if int(t)>=2:
        meal.iloc[i,3]='Meal_Abnormal'
        
sleep['label']='Sleep_Normal'
for i,t in enumerate(sleep['BeginTimeH']):
    if int(t)<=3:
        sleep.iloc[i,3]='Sleep_Abnormal'
for i,t in enumerate(sleep['DurationH']):
    if int(t)>=11:
        sleep.iloc[i,3]='Sleep_Abnormal'
        
toilet['label']='Toilet_Normal'
for i,t in enumerate(toilet['BeginTimeH']):
    if 0<=int(t)<6:
        toilet.iloc[i,3]='Toilet_Abnormal'
        
# stairs['label']='stairs_Normal'
# for i,t in enumerate(stairs['DurationM']):
#     if int(t)>15:
#         stairs.iloc[i,3]='stairs_Abnormal'

In [12]:
meal['label'].value_counts()

Meal_normal      109
Meal_Abnormal      8
Name: label, dtype: int64

In [13]:
sleep['label'].value_counts()

Sleep_Normal      59
Sleep_Abnormal    11
Name: label, dtype: int64

In [14]:
# stairs['label'].value_counts()

In [15]:
toilet['label'].value_counts()

Toilet_Normal      79
Toilet_Abnormal    20
Name: label, dtype: int64

In [16]:
dis=meal.append(sleep).append(toilet)#.append(stairs)
df5=df4.iloc[:,[0,2,75]]
df5['label']='Normal'
for e,i in enumerate(dis.index):
    df5.iloc[i,3]=dis.iloc[e,3]

In [17]:
Meal=meal.iloc[:,[1,2,3]]
Sleep=sleep.iloc[:,[0,1,2,3]]
Toilet=toilet.iloc[:,[0,2,3]]
# stairs=stairs.iloc[:,[1,2,3]]

In [18]:
for e,i in enumerate(df5['Activity']):
    if i =='Meal_Preparation':
        df5.iloc[e,2]=0
    elif i =='R1_Bed_to_Toilet'or i =='R2_Bed_to_Toilet':
        df5.iloc[e,2]=1
    elif i =='R1_Sleep'or i =='R2_Sleep':
        df5.iloc[e,2]=2
#     elif i =='up&downstairs':
#         df5.iloc[e,2]=3   
    else:
        df5.iloc[e,2]=3
    

In [19]:
Values= df5.iloc[:,0:3].values
Target = df5.iloc[:,3].values

In [20]:
Values, Target = shuffle(Values,Target)
x_Train, x_Test, y_Train, y_Test = train_test_split(Values, Target, train_size=0.8, random_state=0)

In [21]:
print(x_Train.shape, x_Test.shape, len(y_Train), len(y_Test))

print('Train set: {}  | Test set: {}'.format(round(((len(y_Train)*1.0)/len(Values)),3),
                                                       round((len(y_Test)*1.0)/len(Values),3)))

(411, 3) (103, 3) 411 103
Train set: 0.8  | Test set: 0.2


In [22]:
counts=Counter(Target)
print(counts)# show the distribution of sample

Counter({'Normal': 228, 'Meal_normal': 109, 'Toilet_Normal': 79, 'Sleep_Normal': 59, 'Toilet_Abnormal': 20, 'Sleep_Abnormal': 11, 'Meal_Abnormal': 8})


In [23]:
clf=RandomForestClassifier(n_estimators=150)

#Train the model using the training sets
clf.fit(x_Train,y_Train)

# prediction on test set
Abnormaly_pred_RandomForest=clf.predict(x_Test)

print("Random Forest test for Anormaly Accuracy:", accuracy_score(y_Test, Abnormaly_pred_RandomForest))

Random Forest test for Anormaly Accuracy: 0.9902912621359223


In [24]:
feature_names=['BeginTimeH','DurationH','Activity']
target_names=['Normal', 'Meal_normal', 'Toilet_Normal', 'Sleep_Normal', 'Toilet_Abnormal', 'Sleep_Abnormal', 'Meal_Abnormal']

In [25]:
def visualise_tree(tree_to_print,index):
    plt.figure()
    fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (40,10), dpi=100)
    tree.plot_tree(tree_to_print,
               feature_names = feature_names,
               class_names=target_names, 
               filled = True,
              rounded=True);
    plt.savefig('figure/'+str(index)+'.jpg')
# for index in range(0, 5):
#     visualise_tree(clf.estimators_[index],index)

In [26]:

# 33561165565 33561165565 55535665 3235321121 32132 35615 23523165 612 312165
i=0
for act in Abnormaly_pred_RandomForest:
    if 'Abnormal' in act:
        duration =1000
        freq =440 
        i+=1
        winsound.Beep(freq,duration)
print('There are '+str(i)+' abnomaly detected!')

There are 11 abnomaly detected!
